In [ ]:
import xarray as xr
import cfgrib
%pylab inline
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')
from wrf import smooth2d


In [ ]:
def make_cmap(n, name='tab20'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return get_cmap(name, n)

def get_slp(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'meanSea'}})
    var = file.get('prmsl').to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var


In [ ]:
# min_lat = 27
# max_lat = 41
# min_lon = -90
# max_lon = -70
# nens = 20
# latind = np.zeros(nens,dtype=np.int8)
# lonind = np.zeros(nens,dtype=np.int8)
# for i in range(1,nens+1):
#     if i < 10:
#         filename = '/p/work1/lloveras/real/nov2018/gefs_data/forecast/init_nov11_18z/gep0'+str(i)+'.t18z.pgrb2af96'
#     else:
#         filename = '/p/work1/lloveras/real/nov2018/gefs_data/forecast/init_nov11_18z/gep'+str(i)+'.t18z.pgrb2af96'
#     slp = get_slp(filename, min_lat, max_lat, min_lon, max_lon)
#     latmin, lonmin = np.unravel_index(np.argmin(np.asarray(slp['prmsl'])), np.asarray(slp['prmsl']).shape)
#     latind[i-1] = int(latmin)
#     lonind[i-1] = int(lonmin)
    
# slp_ctl = get_slp('/p/work1/lloveras/real/nov2018/gefs_data/forecast/init_nov11_18z/gec00.t18z.pgrb2af96',
#                   min_lat, max_lat, min_lon, max_lon)
# latmin_ctl, lonmin_ctl = np.unravel_index(np.argmin(np.asarray(slp_ctl['prmsl'])), np.asarray(slp_ctl['prmsl']).shape)

# slp_anl = get_slp('/p/work1/lloveras/real/nov2018/gefs_data/analysis/valid_nov15_18z/gec00.t18z.pgrb2aanl',
#                   min_lat, max_lat, min_lon, max_lon)
# latmin_anl, lonmin_anl = np.unravel_index(np.argmin(np.asarray(slp_anl['prmsl'])), np.asarray(slp_anl['prmsl']).shape)

# min_lat = 25
# max_lat = 37
# min_lon = -92
# max_lon = -77
# nens = 20
# latind = np.zeros(nens,dtype=np.int8)
# lonind = np.zeros(nens,dtype=np.int8)
# for i in range(1,nens+1):
#     if i < 10:
#         filename = '/p/work1/lloveras/real/dec2018/gefs_data/forecast/init_dec8_12z/gep0'+str(i)+'.t12z.pgrb2af24'
#     else:
#         filename = '/p/work1/lloveras/real/dec2018/gefs_data/forecast/init_dec8_12z/gep'+str(i)+'.t12z.pgrb2af24'
#     slp = get_slp(filename, min_lat, max_lat, min_lon, max_lon)
#     latmin, lonmin = np.unravel_index(np.argmin(np.asarray(slp['prmsl'])), np.asarray(slp['prmsl']).shape)
#     latind[i-1] = int(latmin)
#     lonind[i-1] = int(lonmin)
    
# slp_ctl = get_slp('/p/work1/lloveras/real/dec2018/gefs_data/forecast/init_dec8_12z/gec00.t12z.pgrb2af24',
#                   min_lat, max_lat, min_lon, max_lon)
# latmin_ctl, lonmin_ctl = np.unravel_index(np.argmin(np.asarray(slp_ctl['prmsl'])), np.asarray(slp_ctl['prmsl']).shape)

# slp_anl = get_slp('/p/work1/lloveras/real/dec2018/gefs_data/analysis/valid_dec9_12z/gec00.t12z.pgrb2aanl',
#                   min_lat, max_lat, min_lon, max_lon)
# latmin_anl, lonmin_anl = np.unravel_index(np.argmin(np.asarray(slp_anl['prmsl'])), np.asarray(slp_anl['prmsl']).shape)

min_lat = 30
max_lat = 45
min_lon = -80
max_lon = -60
latind = np.zeros(nens,dtype=np.int8)
lonind = np.zeros(nens,dtype=np.int8)
for i in range(1,nens+1):
    if i < 10:
        filename = '/p/work1/lloveras/real/jan2022/gefs_data/forecast/init_jan25_12z/gep0'+str(i)+'.t12z.pgrb2a.0p50.f096'
    else:
        filename = '/p/work1/lloveras/real/jan2022/gefs_data/forecast/init_jan25_12z/gep'+str(i)+'.t12z.pgrb2a.0p50.f096'
    slp = get_slp(filename, min_lat, max_lat, min_lon, max_lon)
    latmin, lonmin = np.unravel_index(np.argmin(np.asarray(slp['prmsl'])), np.asarray(slp['prmsl']).shape)
    latind[i-1] = int(latmin)
    lonind[i-1] = int(lonmin)
    
slp_ctl = get_slp('/p/work1/lloveras/real/jan2022/gefs_data/forecast/init_jan25_12z/gec00.t12z.pgrb2a.0p50.f096',
                  min_lat, max_lat, min_lon, max_lon)
latmin_ctl, lonmin_ctl = np.unravel_index(np.argmin(np.asarray(slp_ctl['prmsl'])), np.asarray(slp_ctl['prmsl']).shape)

slp_anl = get_slp('/p/work1/lloveras/real/jan2022/gefs_data/analysis/valid_jan29_12z/gec00.t12z.pgrb2a.0p50.f000',
                  min_lat, max_lat, min_lon, max_lon)
latmin_anl, lonmin_anl = np.unravel_index(np.argmin(np.asarray(slp_anl['prmsl'])), np.asarray(slp_anl['prmsl']).shape)


In [ ]:
# Create a figure
fig = plt.figure(figsize=(12,6))
ax = plt.axes(projection=crs.PlateCarree())

# Plot the data

cmap = make_cmap(nens)
for i in range(nens):
    ax.scatter(slp['longitude'][lonind[i]], slp['latitude'][latind[i]],
               transform=crs.PlateCarree(),color=cmap(i),alpha=1.0,s=75,zorder=1)

ax.scatter(slp['longitude'][lonmin_anl], slp['latitude'][latmin_anl],
           transform=crs.PlateCarree(),color='r',alpha=1.0,marker='*',s=200,zorder=1)

ax.scatter(slp['longitude'][lonmin_ctl], slp['latitude'][latmin_ctl],
           transform=crs.PlateCarree(),color='blue',alpha=1.0,s=75,zorder=1)

cs1 = ax.contour(slp_anl['longitude'], slp_anl['latitude'], slp_anl['prmsl']/100,
                  levels=np.arange(902,1060,8),
                  colors='k',alpha=0.7,transform=crs.PlateCarree(),zorder=-1)
ax.clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(902,1060,8),fontsize=6,colors='k',zorder=-1)

# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces")
ax.add_feature(states, linewidth=.5, edgecolor="grey")
ax.coastlines('50m', linewidth=0.8,color='grey',zorder=-1)

# Add the gridlines
gls = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed',zorder=0)
gls.top_labels=False

plt.title("SLP Valid 12Z 29 Jan 2022")

plt.show()
